In [90]:
kittens = open('./example.in')

In [91]:
lines = kittens.readlines()

In [92]:
lines = [l.strip() for l in lines]
current_line = 0

In [93]:
videos_count, endpoints_count, requests_descriptions_count, caches_count, cache_size = [int(l) for l in lines[current_line].split(' ')]
current_line += 1

In [94]:
video_sizes = [int(l) for l in lines[current_line].split()]
current_line += 1

In [95]:
videos_map = dict()
for i in range(videos_count):
    videos_map[i] = video_sizes[i]

latencies = dict()

In [96]:
for ec in range(endpoints_count):
    endpoint_dc_latency, caches_amount = [int(l) for l in lines[current_line].split()]
    current_line += 1
    latencies[(ec, -1)] = endpoint_dc_latency

    for ca in range(caches_amount):
        cache_index, latency = [int(l) for l in lines[current_line].split()]
        current_line += 1

        latencies[(ec, cache_index)] = latency


In [97]:
requests_list = []
for i in range(requests_descriptions_count):
    video_id, endpoint_id, requests_count = [int(l) for l in lines[current_line].split()]
    current_line += 1

    requests_list.append((video_id, endpoint_id, requests_count))

In [98]:
requests_list

[(3, 0, 1500), (0, 1, 1000), (4, 0, 500), (1, 0, 1000)]

In [118]:
endpoints_to_sources = dict()

for (endpoint, source_id),v in latencies.items():
    if endpoint in endpoints_to_sources:
        endpoints_to_sources[endpoint].add(source_id)
    else:
        endpoints_to_sources[endpoint] = set([source_id])

In [121]:
caches_to_videos = dict()